In [2]:
import pandas as pd

#Carmen, deepGeo, deepGeo+Noise, GeoLocText, GeoLocLP, GeoLocLP+Text+CEL
tool="GeoLocLP+Text+CEL"

geoToolResults = 'C:/Users/Dennis/Desktop/BachelorArbeit/resultsALL/'+tool+'.csv'
tweetCOV='C:/Users/Dennis/Desktop/BachelorArbeit/TweetsCOV19.tsv'
df_tweetCOV = pd.read_csv(tweetCOV, delimiter ='\t',names=['id','username',"timestamp","Follower","Friends","Retweets","Favorites","Entities","Sentiment","Mentions","Hashtags","URLs"])
#print(len(df_tweetCOV["id"].unique()))

8077794


In [3]:
# Merge results with TweetsCOV19.tsv to gain sentiment information for tweets etc.
df_tweetCOV = pd.read_csv(tweetCOV, delimiter ='\t',names=['id','username',"timestamp","Follower","Friends","Retweets","Favorites","Entities","Sentiment","Mentions","Hashtags","URLs"])

if tool=="Carmen":
    df_tool = pd.read_csv(geoToolResults, delimiter =';',names=['id','latitude',"longitude","country","state","county","city","known","location_id"])
    df_merged=pd.merge(df_tweetCOV,df_tool, left_on='id', right_on='id', how='inner')

elif tool =="GeoLocText" or "GeoLocLP" or "GeoLocLP+Text+CEL":
    df_tool = pd.read_csv(geoToolResults, delimiter =',',names=['id','country',"state","city"], encoding='cp1252')
    df_merged=pd.merge(df_tweetCOV,df_tool, left_on='id', right_on='id', how='inner')

else: 
    df_tool = pd.read_csv(geoToolResults, delimiter =';',names=['id','country',"state","city"], encoding='cp1252')
    df_merged=pd.merge(df_tweetCOV,df_tool, left_on='id', right_on='id', how='inner')

In [4]:
list_countries = ["United States","United Kingdom","Germany","France","Spain","India","Italy"]

# Show number of tweets for list of countries
for x in list_countries:
    df = df_merged[df_merged["country"]==x]
    print(x+" "+str(len(df)))

United States 3374343
United Kingdom 1023228
Germany 23396
France 29244
Spain 69890
India 313297
Italy 27662
null 0


In [ ]:
#************************************************************************************************

In [ ]:
#************************************************************************************************

In [251]:
# Tweets Countries Breakdown
dataCountries={}

region='country'

for x in df_merged[region].unique():
    dataCountries[x]=len(df_merged[df_merged[region]==x])

gt1m=0
gt100k=0
gt50k=0
gt10k=0
gt5k=0
gt1k=0


for x,y in dataCountries.items():
    if x=="null" or (x is None):
        continue
    if y>1000000:
        gt1m=gt1m+1
    elif y>100000:
        gt100k=gt100k+1
    elif y>50000:
        gt50k=gt50k+1
    elif y>10000:
        gt10k=gt10k+1
    elif y>5000:
        gt5k=gt5k+1
    elif y>1000:
        gt1k=gt1k+1

unique = len(df_merged[region].unique())
print("Countries Breakdown")
print("Number uniqe Countries: "+str(unique))
print(str(gt1m) +" > 1M")
print(str(gt100k) +" > 100K")
print(str(gt50k) +" > 50K")
print(str(gt10k) +" > 10K")
print(str(gt5k) +" > 5K")
print(str(gt1k) +" > 1K")

Countries Breakdown
Number uniqe Countries: 184
1 > 1M
3 > 100K
3 > 50K
10 > 10K
9 > 5K
30 > 1K


In [252]:
# Tweets Cities Breakdown
dataCities={}
region='city'

for x in df_merged[region].unique():
    dataCities[x]=len(df_merged[df_merged[region]==x])

gt100k=0
gt50k=0
gt10k=0
gt5k=0
gt1k=0
gt500=0
gt100=0

for x,y in dataCities.items():
    if x=="null" or (x is None):
        continue
    if y>100000:
        gt100k=gt100k+1
    elif y>50000:
        gt50k=gt50k+1
    elif y>10000:
        gt10k=gt10k+1
    elif y>5000:
        gt5k=gt5k+1
    elif y>1000:
        gt1k=gt1k+1
    elif y>500:
        gt500=gt500+1
    elif y>100:
        gt100=gt100+1   

unique = len(df_merged[region].unique())
print("Cities Breakdown")
print("Number uniqe Cities: "+str(unique))
print(str(gt100k) +" > 100K")
print(str(gt50k) +" > 50K")
print(str(gt10k) +" > 10K")
print(str(gt5k) +" > 5K")
print(str(gt1k) +" > 1K")
print(str(gt500) +" > 500")
print(str(gt100) +" > 100")

Cities Breakdown
Number uniqe Cities: 9165
2 > 100K
2 > 50K
49 > 10K
68 > 5K
496 > 1K
385 > 500
1474 > 100


In [253]:
# Analyse Average Retweet, Like, Follower per Sentiment(pos + neg Score) for specific country

In [254]:
def analyse_Sentiment(country):
    df_country = df_merged[df_merged["country"]==country]

    sentCount={}
    retweetPerTweet={}
    followerPerTweet={}
    favoritePerTweet={}
    count=0
    for index,line in df_country.iterrows():
        sentiment=line["Sentiment"]
        retweet=int(line["Retweets"])
        follower=int(line["Follower"])
        favorites=int(line["Favorites"])

        sentSplit=sentiment.split(" ")
        sentSum=int(sentSplit[0])+int(sentSplit[1])
        strSent=str(sentSum)

        if strSent not in sentCount:
            sentCount[strSent]=1
            retweetPerTweet[strSent]=retweet
            followerPerTweet[strSent]=follower
            favoritePerTweet[strSent]=favorites
        else:
            sentCount[strSent]=sentCount[strSent]+1
            retweetPerTweet[strSent]=retweetPerTweet[strSent]+retweet
            followerPerTweet[strSent]=followerPerTweet[strSent]+follower
            favoritePerTweet[strSent]=favoritePerTweet[strSent]+favorites
        count=count+1

    sorted_sentCount=dict(sorted(sentCount.items(),key = lambda x: int(x[0])))
    sorted_retweet=dict(sorted(retweetPerTweet.items(),key = lambda x: int(x[0])))
    sorted_follower=dict(sorted(followerPerTweet.items(),key = lambda x: int(x[0])))
    sorted_favorite=dict(sorted(favoritePerTweet.items(),key = lambda x: int(x[0])))

    for key,value in sorted_sentCount.items():
        avgRetweet=round(sorted_retweet[key]/value,2)
        avgFollower=round(sorted_follower[key]/value,2)
        avgFavorite=round(sorted_favorite[key]/value,2)
        percentage = round(((value/count)*100),2)
        print("Sentiment "+key+" : Percentage Tweets "+str(percentage)+"%,  Retweet "+str(avgRetweet)+", Likes "+str(avgFavorite)+", Follower "+str(avgFollower))

In [255]:
#Analyse Pro Land
analyse_Sentiment("United States")

Sentiment -4 : Percentage Tweets 0.19%,  Retweet 57.09, Likes 215.98, Follower 132420.41
Sentiment -3 : Percentage Tweets 3.69%,  Retweet 52.95, Likes 172.42, Follower 276020.31
Sentiment -2 : Percentage Tweets 8.78%,  Retweet 52.38, Likes 170.35, Follower 319537.75
Sentiment -1 : Percentage Tweets 11.74%,  Retweet 46.29, Likes 156.74, Follower 272113.65
Sentiment 0 : Percentage Tweets 45.78%,  Retweet 41.35, Likes 144.15, Follower 331521.07
Sentiment 1 : Percentage Tweets 20.59%,  Retweet 37.37, Likes 137.62, Follower 182474.64
Sentiment 2 : Percentage Tweets 8.46%,  Retweet 37.15, Likes 137.6, Follower 228007.26
Sentiment 3 : Percentage Tweets 0.74%,  Retweet 34.15, Likes 135.13, Follower 165394.73
Sentiment 4 : Percentage Tweets 0.03%,  Retweet 34.42, Likes 144.78, Follower 55603.63


In [256]:
analyse_Sentiment("Germany")

Sentiment -4 : Percentage Tweets 0.28%,  Retweet 93.89, Likes 111.67, Follower 32488.33
Sentiment -3 : Percentage Tweets 3.04%,  Retweet 28.94, Likes 56.7, Follower 28014.35
Sentiment -2 : Percentage Tweets 8.27%,  Retweet 23.96, Likes 92.68, Follower 85615.25
Sentiment -1 : Percentage Tweets 11.09%,  Retweet 35.02, Likes 136.37, Follower 65686.16
Sentiment 0 : Percentage Tweets 45.74%,  Retweet 22.36, Likes 123.33, Follower 172511.02
Sentiment 1 : Percentage Tweets 22.49%,  Retweet 31.69, Likes 204.39, Follower 74284.45
Sentiment 2 : Percentage Tweets 8.3%,  Retweet 25.99, Likes 100.4, Follower 87471.1
Sentiment 3 : Percentage Tweets 0.78%,  Retweet 12.72, Likes 60.28, Follower 82358.6


In [257]:
analyse_Sentiment("United Kingdom")

Sentiment -4 : Percentage Tweets 0.24%,  Retweet 47.98, Likes 163.72, Follower 141337.94
Sentiment -3 : Percentage Tweets 4.29%,  Retweet 44.85, Likes 147.36, Follower 294238.38
Sentiment -2 : Percentage Tweets 9.67%,  Retweet 47.5, Likes 157.69, Follower 360818.65
Sentiment -1 : Percentage Tweets 13.41%,  Retweet 44.02, Likes 140.48, Follower 303913.8
Sentiment 0 : Percentage Tweets 40.86%,  Retweet 36.52, Likes 122.34, Follower 512898.0
Sentiment 1 : Percentage Tweets 21.54%,  Retweet 33.89, Likes 124.16, Follower 236962.61
Sentiment 2 : Percentage Tweets 9.09%,  Retweet 29.57, Likes 125.17, Follower 248711.97
Sentiment 3 : Percentage Tweets 0.87%,  Retweet 26.62, Likes 104.11, Follower 168668.12
Sentiment 4 : Percentage Tweets 0.04%,  Retweet 16.38, Likes 101.11, Follower 515749.31


In [258]:
analyse_Sentiment("France")

Sentiment -4 : Percentage Tweets 0.13%,  Retweet 25.0, Likes 71.25, Follower 27307.75
Sentiment -3 : Percentage Tweets 3.28%,  Retweet 44.06, Likes 131.04, Follower 186406.85
Sentiment -2 : Percentage Tweets 8.5%,  Retweet 65.75, Likes 247.02, Follower 205358.99
Sentiment -1 : Percentage Tweets 12.3%,  Retweet 40.25, Likes 137.33, Follower 143381.54
Sentiment 0 : Percentage Tweets 45.1%,  Retweet 38.58, Likes 124.08, Follower 197205.15
Sentiment 1 : Percentage Tweets 21.02%,  Retweet 33.79, Likes 109.59, Follower 131065.18
Sentiment 2 : Percentage Tweets 8.92%,  Retweet 34.04, Likes 133.88, Follower 173565.94
Sentiment 3 : Percentage Tweets 0.73%,  Retweet 33.14, Likes 146.25, Follower 125992.4
Sentiment 4 : Percentage Tweets 0.02%,  Retweet 26.25, Likes 76.5, Follower 3098.0


In [259]:
analyse_Sentiment("Italy")

Sentiment -4 : Percentage Tweets 0.26%,  Retweet 15.31, Likes 52.62, Follower 208930.15
Sentiment -3 : Percentage Tweets 3.73%,  Retweet 48.01, Likes 120.07, Follower 146047.85
Sentiment -2 : Percentage Tweets 8.74%,  Retweet 48.06, Likes 140.7, Follower 197424.56
Sentiment -1 : Percentage Tweets 11.99%,  Retweet 25.39, Likes 92.02, Follower 233423.03
Sentiment 0 : Percentage Tweets 43.56%,  Retweet 40.12, Likes 146.32, Follower 212212.1
Sentiment 1 : Percentage Tweets 20.53%,  Retweet 27.2, Likes 112.89, Follower 242903.36
Sentiment 2 : Percentage Tweets 10.29%,  Retweet 93.45, Likes 279.48, Follower 270566.72
Sentiment 3 : Percentage Tweets 0.89%,  Retweet 35.96, Likes 220.96, Follower 165180.4
Sentiment 4 : Percentage Tweets 0.02%,  Retweet 0.0, Likes 0.0, Follower 334.0


In [260]:
analyse_Sentiment("India")

Sentiment -4 : Percentage Tweets 0.21%,  Retweet 76.91, Likes 235.08, Follower 356754.85
Sentiment -3 : Percentage Tweets 4.16%,  Retweet 73.3, Likes 185.65, Follower 536066.24
Sentiment -2 : Percentage Tweets 10.49%,  Retweet 66.12, Likes 184.51, Follower 552798.54
Sentiment -1 : Percentage Tweets 13.55%,  Retweet 62.07, Likes 181.77, Follower 514328.11
Sentiment 0 : Percentage Tweets 41.28%,  Retweet 48.11, Likes 152.72, Follower 744433.67
Sentiment 1 : Percentage Tweets 20.72%,  Retweet 53.34, Likes 182.89, Follower 453030.94
Sentiment 2 : Percentage Tweets 8.97%,  Retweet 54.73, Likes 194.09, Follower 595719.7
Sentiment 3 : Percentage Tweets 0.6%,  Retweet 54.45, Likes 208.78, Follower 403512.43
Sentiment 4 : Percentage Tweets 0.03%,  Retweet 129.16, Likes 555.09, Follower 129616.17


In [261]:
analyse_Sentiment("Spain")

Sentiment -4 : Percentage Tweets 0.23%,  Retweet 44.05, Likes 77.93, Follower 109161.17
Sentiment -3 : Percentage Tweets 4.48%,  Retweet 50.71, Likes 112.98, Follower 155706.27
Sentiment -2 : Percentage Tweets 11.13%,  Retweet 52.86, Likes 122.73, Follower 249845.09
Sentiment -1 : Percentage Tweets 13.36%,  Retweet 50.9, Likes 123.26, Follower 233447.52
Sentiment 0 : Percentage Tweets 43.17%,  Retweet 48.61, Likes 149.81, Follower 347729.39
Sentiment 1 : Percentage Tweets 19.83%,  Retweet 49.96, Likes 171.41, Follower 340558.6
Sentiment 2 : Percentage Tweets 7.18%,  Retweet 50.87, Likes 168.25, Follower 341919.57
Sentiment 3 : Percentage Tweets 0.61%,  Retweet 42.29, Likes 138.85, Follower 538568.78
Sentiment 4 : Percentage Tweets 0.02%,  Retweet 25.82, Likes 216.05, Follower 69201.64


In [262]:
#************************************************************************************************

In [263]:
#************************************************************************************************

In [3]:
# Analysis Sentiment + plot

In [4]:
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
from datetime import datetime
import matplotlib.ticker as tck

def get_Sentiment(country):
    #Set DataFrame to specific Country
    df_country = df_merged[df_merged["country"]==country]
    
    #Gather all positive/negative scores per Day
    pos_score_per_day={}
    neg_score_per_day={}
    
    #Number of Tweets per Day
    countPerDay={}

    
    for index,line in df_country.iterrows():
        #Get TimeStamp,Sentiment Columns

        time=line["timestamp"]
        sentiment=line["Sentiment"]  
        
        #Split to get only Month + Day + Year
        time = time.split(" ")
        modTime=str(time[1]+" "+time[2]+" "+time[5])
        
        #We will not consider september
        if time[1]=="Sep":
            continue
            
        #Sentiment is seperated into two values, positive score and negative score    
        sentSplit=sentiment.split(" ")
        posSent=int(sentSplit[0])
        negSent=int(sentSplit[1])
        

        if modTime not in pos_score_per_day:
            pos_score_per_day[modTime]=posSent
            neg_score_per_day[modTime]=negSent
            countPerDay[modTime]=1
        else:
            pos_score_per_day[modTime]=pos_score_per_day[modTime]+posSent
            neg_score_per_day[modTime]=neg_score_per_day[modTime]+negSent
            countPerDay[modTime]=countPerDay[modTime]+1
    
    #Sort dates 
    dates = list(countPerDay.keys())
    dates = sorted(dates, key = lambda date: datetime.strptime(date ,"%b %d %Y"))
    
    pos_plot=[]
    neg_plot=[]
    avg_plot=[]
    list_days=[]
    
    #Fill list_days so we are able to show only "Month"+"Day" in Plot
    #Calculate positive/negative/average score per day
    for day in dates:
        var=day.split(" ")
        list_days.append(var[0]+" "+var[1])
        calc_pos=pos_score_per_day[day]/countPerDay[day]
        calc_neg=neg_score_per_day[day]/countPerDay[day]
        pos_plot.append(calc_pos)
        neg_plot.append(calc_neg)
        avg_plot.append((calc_pos+calc_neg)/2)
    
    #Plot
    fig, ax = plt.subplots(figsize=(14,2))
    ax.plot(list_days,pos_plot)
    ax.plot(list_days,neg_plot)
    ax.plot(list_days,avg_plot)
    ax.set_yticks([-3,-2,-1,0,1,2,3])
    ax.xaxis.set_major_locator(tck.MultipleLocator(base=14.0))
    #ax.axhline(0, color='black', lw=1)
    plt.savefig('C:/Users/dennis/Desktop/'+country+tool+'.png')
    plt.show()

In [ ]:
get_Sentiment("United States")

In [ ]:
get_Sentiment("United Kingdom")

In [ ]:
get_Sentiment("Germany")

In [ ]:
get_Sentiment("France")

In [ ]:
get_Sentiment("Spain")

In [ ]:
get_Sentiment("Italy")

In [ ]:
get_Sentiment("India")

In [ ]:
#************************************************************************************************

In [ ]:
#************************************************************************************************

In [8]:
# Tweet per Country Worldmap

In [3]:
import csv
import pandas as pd
import json,requests, ipyleaflet
from ipyleaflet import Map, GeoJSON, LayersControl, Choropleth, Popup, LegendControl
from ipywidgets import Label, HTML
from branca.colormap import linear
import branca.colormap as color

In [4]:
world_geo ="C:/Users/dennis/Desktop/BachelorArbeit/worldGeo2.json"
with open(world_geo, encoding ="utf-8") as l:
    world_data = json.load(l)

In [5]:
#Replace Country Names, so it matches with the names in GeoJSON
old = ["The Netherlands","Republic of Latvia","Republic of Lithuania","Hong Kong","Democratic Republic of Congo",
       "Macau-China","United States","Serbia","Vatican City","The Gambia","Tanzania","St. Lucia",
       "South Georgia and the South Sandwich Islands","Congo","Russian Federation","Syrian Arab Republic",
       "Bahama","Côte D'ivoire (Ivory Coast)","Islamic Republic of Iran","Korea, Democratic People's Republic of",
       'Korea, Republic of',"Libyan Arab Jamahiriya","Macau",'Moldova, Republic of',"North Macedonia",
       'Trinidad & Tobago','Taiwan, Province of China','Tanzania, United Republic of',"Viet Nam","Antigua & Barbuda",
       "CÃ´te D'ivoire (Ivory Coast)","Islamic Republic of Iran","Lao People's Democratic Republic","Lybia","Micronesia",
       "Pitcairn","Republic of the Congo","Sao Tome & Principe","St. Helena","St. Pierre & Miquelon","St. Vincent & the Grenadines",
       "Turks & Caicos Islands","Cook Iislands","Guinea-Bissau"]
    

new = ["Netherlands", "Latvia", "Lithuania", "Hong Kong S.A.R.", "Democratic Republic of the Congo",
       "Macao S.A.R","United States of America","Republic of Serbia","Vatican","Gambia",
       "United Republic of Tanzania","Saint Lucia","South Georgia and South Sandwich Islands",
       "Republic of Congo","Russia","Syria","The Bahamas","Ivory Coast","Iran","North Korea","South Korea",
       "Libya","Macao S.A.R","Moldova","Macedonia",'Trinidad and Tobago',"Taiwan",'United Republic of Tanzania',
       "Vietnam","Antigua and Barbuda","Ivory Coast","Iran","Laos","Libya","Federated States of Micronesia",
       "Pitcairn Islands","Republic of Congo","Sao Tome and Principe","Saint Helena","Saint Pierre and Miquelon",
       "Saint Vincent and the Grenadines","Turks and Caicos Islands","Cook Islands","Guinea Bissau"]

In [6]:
df_merged_world= df_merged.replace(old, new )

In [7]:
#Get all countries found by GeoTool
df_countries = df_merged_world[df_merged_world["country"]!="null"]
countries = df_countries["country"].unique()
countries = list(countries)
countries = [country for country in countries if str(country)!='nan']
list_countries = sorted(countries)
print(len(list_countries))
print(list_countries)

184
['Afghanistan', 'Albania', 'Algeria', 'Angola', 'Anguilla', 'Antigua and Barbuda', 'Argentina', 'Aruba', 'Australia', 'Austria', 'Azerbaijan', 'Bahrain', 'Bangladesh', 'Barbados', 'Belarus', 'Belgium', 'Benin', 'Bermuda', 'Bolivia', 'Bosnia and Herzegovina', 'Botswana', 'Brazil', 'Bulgaria', 'Burkina Faso', 'Cambodia', 'Cameroon', 'Canada', 'Cape Verde', 'Cayman Islands', 'Central African Republic', 'Chad', 'Chile', 'China', 'Christmas Island', 'Cocos (Keeling) Islands', 'Colombia', 'Cook Islands', 'Costa Rica', 'Croatia', 'Cuba', 'Cyprus', 'Czech Republic', 'Denmark', 'Dominican Republic', 'Ecuador', 'Egypt', 'El Salvador', 'Equatorial Guinea', 'Eritrea', 'Estonia', 'Ethiopia', 'Federated States of Micronesia', 'Finland', 'France', 'French Guiana', 'French Polynesia', 'French Southern Territories', 'Gabon', 'Georgia', 'Germany', 'Ghana', 'Greece', 'Greenland', 'Guadeloupe', 'Guatemala', 'Guernsey', 'Guinea Bissau', 'Guyana', 'Haiti', 'Honduras', 'Hong Kong S.A.R.', 'Hungary', 'Ice

In [ ]:
# List containing Country Names within GeoJSON
list_id=[]
# List with Countries of GeoJson, which dont match results country Names
noData = []
#Dictionary with Country:TweetCount
count={}

#Add country name to id for each country in GeoJson
for feature in world_data['features']:
    properties = feature['properties']
    feature["id"] = properties["ADMIN"]
    list_id.append(feature["id"])

#Fill found countries in dict with tweet count value
for country in list_countries:
    notInGeo=["Christmas Island","Guadeloupe","Martinique","Netherlands Antilles","Palestinian Territories","Reunion"]
    if country in notInGeo:
        continue   
    df_country = df_merged_world[df_merged_world["country"]== country]

    count[country]=len(df_country)    
    
#Create List of not found countries
for country in list_id:
    if country not in list_countries:
        noData.append(country)
        
#Fill not found countries in dict with value 0        
for country in noData:
    count[country]=0

# Methods for  Click Events, LegendControl

#Get Coordinates
def handle_interaction(**kwargs):
    if kwargs.get('type') == 'mousemove':
        label.value = str(kwargs.get('coordinates'))
    return label.value

# Click event -> PopUp
def my_method(event=None, feature=None, id=None, properties=None): 
    if id in count.keys():
        popup = HTML()
        popup.value = str(id)+" <br> Tweetcount: "+str(count[id])

        coordinates = handle_interaction()
        coordinates = coordinates.replace('[', "")
        coordinates = coordinates.replace(']', "")
        split_coordinate = coordinates.split(", ")
        coordinates = [float(split_coordinate[0]),float(split_coordinate[1])]

        popup = Popup(
            location=coordinates,
            child=popup,
            close_button=True,
            auto_close=True,
            close_on_escape_key=True
        )
        m.add_layer(popup)

colorScheme = ["#FFFCFC", "#F1DCDC","#DB8585", "#BB1216","#6c0000"]
colormap_dept = color.StepColormap(colors=colorScheme,vmin=min(count.values()),vmax=max(count.values()),index=[0, 10000, 50000, 100000,1000000])
legend = LegendControl({"0 - 1K":colorScheme[0], "1K - 50K":colorScheme[1],"50K - 100K":colorScheme[2],"100K - 1M":colorScheme[3],"1M+": colorScheme[4]},fontsize=2, name="Tweets count", position="topright")


In [35]:
#not in geo : Cocos (Keeling) Islands, French Guiana, French Southern Territories, Guadeloupe, Martinique, 
#Netherlands Antilles, RÃ©union, Palestinian Territories,Christmas Island

for country in list_countries:
    if country not in list_id:
        print(country)
print("******************")
print(list_id)
print(count)

Christmas Island
Cocos (Keeling) Islands
French Guiana
French Southern Territories
Guadeloupe
Martinique
Netherlands Antilles
RÃ©union
******************
['Aruba', 'Afghanistan', 'Angola', 'Anguilla', 'Albania', 'Aland', 'Andorra', 'United Arab Emirates', 'Argentina', 'Armenia', 'American Samoa', 'Antarctica', 'Ashmore and Cartier Islands', 'French Southern and Antarctic Lands', 'Antigua and Barbuda', 'Australia', 'Austria', 'Azerbaijan', 'Burundi', 'Belgium', 'Benin', 'Burkina Faso', 'Bangladesh', 'Bulgaria', 'Bahrain', 'The Bahamas', 'Bosnia and Herzegovina', 'Bajo Nuevo Bank (Petrel Is.)', 'Saint Barthelemy', 'Belarus', 'Belize', 'Bermuda', 'Bolivia', 'Brazil', 'Barbados', 'Brunei', 'Bhutan', 'Botswana', 'Central African Republic', 'Canada', 'Switzerland', 'Chile', 'China', 'Ivory Coast', 'Clipperton Island', 'Cameroon', 'Cyprus No Mans Area', 'Democratic Republic of the Congo', 'Republic of Congo', 'Cook Islands', 'Colombia', 'Comoros', 'Cape Verde', 'Costa Rica', 'Coral Sea Island

In [36]:
from ipyleaflet import Map, basemaps

label = Label()
m = Map(layers=(),center=(35.52978589571, 7.84580993652344),zoom_control=False,zoom=2)

layer = ipyleaflet.Choropleth(
    geo_data= world_data,
    choro_data=count,
    key_on="id",
    colormap=colormap_dept,
    style={'fillOpacity': 1}
)

layer.on_click(my_method)
m.on_interaction(handle_interaction)
m.add_layer(layer)
m.add_control(legend)
m.layout.height = '800px'
m

Map(center=[35.52978589571, 7.84580993652344], controls=(AttributionControl(options=['position', 'prefix'], po…

In [17]:
#************************************************************************************************

In [18]:
#************************************************************************************************

In [37]:
# Analyse Retweet, Follower, Likes for Specific country ( Map )

In [38]:
# Methods for  Click Events

#Get Coordinates
def handle_interaction_country(**kwargs):
    if kwargs.get('type') == 'mousemove':
        label.value = str(kwargs.get('coordinates'))
    return label.value

# Click event -> PopUp
def set_popUp(event=None, feature=None, id=None, properties=None): 
    if id in data.keys():
        popup = HTML()
        popup.value = str(id)+" <br> Average: "+str(data[id])

        coordinates = handle_interaction()
        coordinates = coordinates.replace('[', "")
        coordinates = coordinates.replace(']', "")
        split_coordinate = coordinates.split(", ")
        coordinates = [float(split_coordinate[0]),float(split_coordinate[1])]

        popup = Popup(
            location=coordinates,
            child=popup,
            close_button=True,
            auto_close=True,
            close_on_escape_key=True
        )
        m.add_layer(popup)

In [39]:
import json
spain_geo = 'C:/Users/dennis/Desktop/Bachelorarbeit/states/spain/states.geojson'
germany_geo = 'C:/Users/dennis/Desktop/Bachelorarbeit/states/germany/states.geojson'
france_geo='C:/Users/dennis/Desktop/Bachelorarbeit/states/france/states.geojson'
italy_geo = 'C:/Users/dennis/Desktop/Bachelorarbeit/states/italy/states.geojson'
india_geo = 'C:/Users/dennis/Desktop/Bachelorarbeit/states/india/states.geojson'
us_geo = 'C:/Users/dennis/Desktop/Bachelorarbeit/states/united states/states.geojson'
uk_geo = 'C:/Users/dennis/Desktop/Bachelorarbeit/states/united kingdom/states.geojson'


with open(germany_geo,encoding='utf-8') as f:
    german_data = json.load(f)
    
with open(spain_geo, encoding="utf-8")as g:
    spain_data = json.load(g)
    
with open(france_geo,encoding='utf-8') as h:
    france_data = json.load(h)
    
with open(italy_geo,encoding='utf-8') as j:
    italy_data = json.load(j)

with open(india_geo, encoding ="utf-8") as k:
    india_data = json.load(k)

with open(us_geo, encoding ="utf-8") as l:
    us_data = json.load(l)

with open(uk_geo,encoding='utf-8') as f:
    uk_data = json.load(f)    
    
for feature in spain_data['features']:
    properties = feature['properties']
    feature.update(id=properties['name'])
for feature in german_data['features']:
    properties = feature['properties']
    feature.update(id=properties['name'])
for feature in france_data['features']:   
    properties = feature['properties']
    feature.update(id = properties['nom'])
for feature in italy_data['features']:   
    properties = feature['properties']
    feature.update(id = properties['id'])
for feature in india_data['features']:   
    properties = feature['properties']
    feature.update(id = properties['st_nm'])
for feature in us_data['features']:   
    properties = feature['properties']
    feature.update(id = properties['name'])
for feature in uk_data['features']:
    properties = feature['properties']
    feature.update(id = properties['NAME_2'])

In [55]:
# edit states to match geoJson 
def edit_states(country, df_country):
    
    if country == "Spain":
        # Castilla-Leon, Catalonia, Ceuta, Murcia, La Rioja, Baleares, Canarias, Cantabria, Andalucia,
        # Asturias, Valencia, Melilla, Navarra, Galicia, Aragon, Madrid, Extremadura, 
        # Castilla-La Mancha, Pais Vasco
        
        oldRegions=["Andalucía","Andalusia","Comunidad de Madrid","Canary Islands",
                    "Balearic Islands","Castilla-la Mancha","Ciudad Autónoma de Melilla",
                    "Comunidad Valenciana","Islas Baleares","Castilla y Leon","Basque Country",
                    "Castille and Leon","Navarre","Castille-La Mancha","Pais Vaso","Basque"]
        
        newRegions=["Andalucia","Andalucia","Madrid","Canarias","Baleares","Castilla-La Mancha","Melilla","Valencia",
                    "Baleares","Castilla-Leon","Pais Vasco","Castilla-Leon","Navarra","Castilla-La Mancha","Pais Vasco","Pais Vasco"]
        df_country =df_country.replace(oldRegions,newRegions)

    elif country == "Germany":
        # Baden-Württemberg,Bayern, Berlin, Brandenburg, Bremen, Hamburg, Hessen, Mecklenburg-Vorpommern,
        # Niedersachsen, Nordrhein-Westfalen, Rheinland-Pfalz, Saarland, Sachsen-Anhalt, Sachsen,
        # Schleswig-Holstein, Thüringen

        oldRegions=["North Rhine-Westphalia","North-Rhine-Westphalia","Bavaria","Saxony","Hesse",
                    "Lower-Saxony","Baden-Wurttemberg","Thuringen","Baden-Wuerttemberg","Lower Saxony","Saxony-Anhalt",
                    "Thuringia"]
        
        newRegions=["Nordrhein-Westfalen","Nordrhein-Westfalen","Bayern","Sachsen","Hessen",
                    "Niedersachsen","Baden-Württemberg","Thüringen","Baden-Württemberg","Niedersachsen","Sachsen-Anhalt",
                    "Thüringen"]
        df_country=df_country.replace(oldRegions,newRegions)

    elif country == "France":
        # Île-de-France, Centre-Val de Loire, Bourgogne-Franche-Comté, Normandie, Hauts-de-France, Grand Est
        # Pays de la Loire, Bretagne, Nouvelle-Aquitaine, Guadeloupe, Martinique, Guyane, La Réunion, Mayotte, 
        # Occitanie, Auvergne-Rhône-Alpes, Provence-Alpes-Côte d'Azur, Corse

        oldRegions=["Ile-de-France","Nord-Pas-de-Calais","Rhone-Alpes","Aquitaine",
                    "Lorraine","Brittany","Centre","Midi-Pyrénées","Bourgogne","Alsace","Picardie",'Rhone-Alpes',
                    'Basse-Normandie',"Ile-de-France","Limousin","Languedoc-Roussillon","Champagne-Ardenne","Haute-Normandie",
                    "Midi-Pyrenees","Franche-Comte","Poitou-Charentes","Auvergne","Provence-Alpes-Cote d'Azur","Corsica",
                    "Lower Normandy"]
        
        newRegions=["Île-de-France","Hauts-de-France","Auvergne-Rhône-Alpes","Nouvelle-Aquitaine",
                    "Grand Est","Bretagne","Centre-Val de Loire","Occitanie","Bourgogne-Franche-Comté","Grand Est",
                    "Hauts-de-France","Auvergne-Rhône-Alpes","Normandie","Île-de-France","Nouvelle-Aquitaine",
                    "Occitanie","Grand Est","Normandie","Occitanie","Bourgogne-Franche-Comté","Nouvelle-Aquitaine","Auvergne-Rhône-Alpes",
                    "Provence-Alpes-Côte d'Azur","Corse","Normandie"]
        df_country = df_country.replace(oldRegions,newRegions)
        
    elif country =="India":
        # Telangana, Andaman & Nicobar Island, Andhra Pradesh, Arunanchal Pradesh, Assam, Bihar, Chhattisgarh,
        # Daman & Diu, Goa, Gujarat, Haryana, Himachal Pradesh, Jammu & Kashmir, Jharkhand, Karnataka
        # Kerala, Lakshadweep, Madhya Pradesh, Maharashtra, Manipur, Chandigarh, Puducherry,
        # Punjab, Rajasthan, Sikkim, Tamil Nadu , Tripura, Uttar Pradesh, Uttarakhand, West Bengal,
        # Odisha, Dadara & Nagar Havelli, Meghalaya, Mizoram, Nagaland, NCT of Delhi
        
        oldRegions=["Delhi","Jammu and Kashmir","Orissa","NCT","Andaman and Nicobar Islands","Dadra and Nagar Haveli",
                    "Kashmir"]
        
        newRegions=["NCT of Delhi","Jammu & Kashmir","Odisha","NCT of Delhi","Andaman & Nicobar Island",
                   "Dadara & Nagar Havelli","Jammu & Kashmir"]
        df_country= df_country.replace(oldRegions,newRegions)
        
    elif country == "Italy":
        # Abruzzo, Puglia, Basilicata, Campania, Emilia-Romagna, Friuli-Venezia Giulia, Lazio, Liguria
        # Lombardia, Marche, Molise, Piemonte, Sardegna, Sicilia, Toscana, Trentino-Alto Adige, Umbria
        # Valle d'Aosta, Veneto, Calabria
        
        oldRegions=["Latium","Lombardy","Piedmont","Tuscany","Emilia Romagna","Sicily","Abruzzi","Apulia",
                    "Basilicate","Friuli Venezia Giulia","Sardinia","The Marches","Aosta Valley"]
        
        newRegions=["Lazio","Lombardia","Piemonte","Toscana","Emilia-Romagna","Sicilia","Abruzzo","Puglia",
                    "Basilicata","Friuli-Venezia Giulia","Sardegna","Marche","Valle d'Aosta"]
        df_country = df_country.replace(oldRegions,newRegions)
        
    elif country == "United States":
        # Alabama,Alaska, Arizona, Arkansas, California, Colorado, Connecticut, Delaware, District of Columbia,
        # Florida, Georgia, Hawaii, Idaho, Illinois, Indiana, Iowa, Kansas, Kentucky, Louisiana, Maine, Maryland,
        # Massachusetts, Michigan, Minnesota, Mississippi, Missouri, Montana, Nebraska, Nevada, New Hampshire,
        # New Jersey, New Mexico, New York, North Carolina, North Dakota, Ohio, Oklahoma, Oregon, Pennsylvania,
        # , Rhode Island, South Carolina, South Dakota, Tennessee, Texas, Utah, Vermont, Virginia, Washington,
        # West Virginia, Wisconsin, Wyoming, Puerto Rico
        oldRegions=["Washington, D.C","Washington, D.C."]
        newRegions=["District of Columbia","District of Columbia"]
        df_country = df_country.replace(oldRegions,newRegions)
        # MISSING: GUAM, US Virgin Islands, Northern Mariana Islands        

        
    elif country == "United Kingdom":
        oldRegions=["Greater London","Aberdeen City","Borough of Bolton","Borough of Halton","Borough of Oldham","Borough of Stockport",
                   "Borough of Swindon","Borough of Torbay","Borough of Wigan","Bracknell Forest","Bradford","Cheshire West and Chester",
                   "City and Borough of Birmingham","City and Borough of Leeds","City and Borough of Salford","City and Borough of Wakefield",
                   "City and County of Swansea","City of Belfast","City of Kingston upon Hull","City of Leicester","County Durham","Coventry",
                   "Dudley","Dundee City","Glasgow City","Kirklees","Liverpool","Metropolitan Borough of Wirral","Newcastle upon Tyne",
                   "Rhondda Cynon Taf","Rotherham","Sefton","Sheffield","Slough","Solihull","South Tyneside","St. Helens","Trafford","Wolverhampton",
                   "City of Bristol","Cleveland","Cornwall and Isles of Scilly","County Antrim","County Down","County Fermanagh","Grampian",
                   "Greater Manchester","Londonderry","Middlesex","Pembrokeshire County","Powys County","Rhondda Cynon Taff County Borough",
                   "Birmingham","Blackpool","Calderdale","Central Bedfordshire","Cheshire East","Gateshead","Leeds","Oldham","Reading",
                   "Rhondda Cynon Taff","Sandwell","Stockport","Sunderland","Wakefield","Walsall","Wigan","Wirral","Chesire","Barnsley",
                   "Bedford","Borough of Bury","Borough of North Tyneside","Borough of Rochdale","Borough of Tameside","Bridgend county borough",
                   "Caerphilly County Borough","Chesire","City of York","County of Ceredigion","County of Flintshire","District of Rutland",
                   "Doncaster","Knowsley","Royal Borough of Windsor and Maidenhead","Sir Powys","The Scottish Borders","Torfaen County Borough",
                   "West Berkshire","Wokingham","Stirlingshire","Strathclyde","West Glamorgan"]
        
        newRegions=["London","Aberdeen","Manchester","Chesire","Manchester","Manchester","Swindon","Torbay","Manchester","Berkshire",
                   'West Yorkshire',"Cheshire","West Midlands","West Yorkshire","Manchester","West Yorkshire","Swansea","Belfast","East Riding of Yorkshire",
                   "Leicestershire","Durham","West Midlands","West Midlands","Dundee","Glasgow","West Yorkshire","Merseyside","Merseyside","Tyne and Wear",
                   "Rhondda, Cynon, Taff","South Yorkshire","Merseyside","South Yorkshire","Berkshire","West Midlands","Tyne and Wear","Merseyside",
                   "Manchester","West Midlands","Bristol","Redcar and Cleveland","Cornwall","Antrim","Down","Fermanagh","Aberdeenshire",
                   "Manchester","Derry","London","Pembrokeshire","Powys","Rhondda, Cynon, Taff","West Midlands","Lancashire","West Yorkshire",
                   "Bedfordshire","Cheshire","Tyne and Wear","West Yorkshire","Manchester","Berkshire","Rhondda, Cynon, Taff","West Midlands",
                   "Manchester","Tyne and Wear","West Yorkshire","West Midlands","Manchester","Merseyside","Cheshire","South Yorkshire",
                   "Bedfordshire","Manchester","Tyne and Wear","Manchester","Manchester","Bridgend","Caerphilly","Cheshire","York","Ceredigion",
                   "Flintshire","Rutland","South Yorkshire","Merseyside","Berkshire","Powys","Scottish Borders","Torfaen","Berkshire","Berkshire",
                   "Stirling","Glasgow","Swansea"]
        df_country = df_country.replace(oldRegions,newRegions)
        
    return df_country

#Fill not Found Counties with Value 0
def fill_states(country,stateCounts):
    
    if country == "Germany":
        regions =["Baden-Württemberg","Bayern", "Berlin", "Brandenburg", "Bremen", "Hamburg", "Hessen", "Mecklenburg-Vorpommern",
                  "Niedersachsen", "Nordrhein-Westfalen", "Rheinland-Pfalz", "Saarland", "Sachsen-Anhalt", "Sachsen",
                  "Schleswig-Holstein", "Thüringen"]
        
    elif country =="Spain":
        regions = [ "Castilla-Leon", "Catalonia", "Ceuta", "Murcia", "La Rioja", "Baleares", "Canarias", "Cantabria", "Andalucia",
                    "Asturias", "Valencia", "Melilla", "Navarra", "Galicia", "Aragon", "Madrid", "Extremadura", 
                    "Castilla-La Mancha", "Pais Vasco"]
                    
    elif country == "France":
        regions =[ "Île-de-France", "Centre-Val de Loire", "Bourgogne-Franche-Comté", "Normandie", "Hauts-de-France", 
                  "Grand Est", "Pays de la Loire", "Bretagne", "Nouvelle-Aquitaine", "Guadeloupe", "Martinique", 
                  "Guyane", "La Réunion", "Mayotte", "Occitanie", "Auvergne-Rhône-Alpes", "Provence-Alpes-Côte d'Azur", "Corse"]
    
    elif country == "Italy":
        regions = [ "Abruzzo", "Basilicata", "Campania", "Emilia-Romagna", "Friuli-Venezia Giulia", "Lazio",
                   "Liguria", "Lombardia", "Marche", "Molise", "Piemonte", "Sardegna", "Sicilia", "Toscana", "Trentino-Alto Adige", 
                   "Umbria", "Valle d'Aosta", "Veneto","Puglia","Calabria"]
        
    elif country =="India":
        regions = ["Telangana", "Andaman & Nicobar Island", "Andhra Pradesh","Arunanchal Pradesh", "Assam", "Bihar", "Chhattisgarh",
                 "Daman & Diu", "Goa", "Gujarat", "Haryana", "Himachal Pradesh", "Jammu & Kashmir", "Jharkhand", "Karnataka",
                 "Kerala", "Lakshadweep", "Madhya Pradesh", "Maharashtra", "Manipur", "Chandigarh", "Puducherry",
                 "Punjab", "Rajasthan", "Sikkim", "Tamil Nadu" , "Tripura", "Uttar Pradesh", "Uttarakhand", "West Bengal",
                 "Odisha", "Dadara & Nagar Havelli", "Meghalaya", "Mizoram", "Nagaland", "NCT of Delhi"]
    
    elif country == "United States":
        regions =["Alabama","Alaska", "Arizona", "Arkansas", "California", "Colorado", "Connecticut", "Delaware", "District of Columbia",
                 "Florida", "Georgia", "Hawaii", "Idaho", "Illinois", "Indiana", "Iowa", "Kansas", "Kentucky", "Louisiana", "Maine", "Maryland",
                 "Massachusetts", "Michigan", "Minnesota", "Mississippi", "Missouri", "Montana", "Nebraska", "Nevada", "New Hampshire",
                 "New Jersey", "New Mexico", "New York", "North Carolina", "North Dakota", "Ohio", "Oklahoma", "Oregon", "Pennsylvania",
                 "Rhode Island", "South Carolina", "South Dakota", "Tennessee", "Texas", "Utah", "Vermont", "Virginia", "Washington",
                 "West Virginia", "Wisconsin", "Wyoming", "Puerto Rico"]
    
    elif country =="United Kingdom":
        regions=['Barking and Dagenham', 'Bath and North East Somerset', 'Bedfordshire', 'Berkshire', 'Bexley', 'Blackburn with Darwen', 
                 'Bournemouth', 'Brent', 'Brighton and Hove', 'Bristol', 'Bromley', 'Buckinghamshire', 'Cambridgeshire', 'Camden', 'Cheshire', 
                 'Cornwall', 'Croydon', 'Cumbria', 'Darlington', 'Derbyshire', 'Derby', 'Devon', 'Dorset', 'Durham', 'Ealing', 
                 'East Riding of Yorkshire', 'East Sussex', 'Enfield', 'Essex', 'Gloucestershire', 'Greenwich', 'Hackney', 'Halton', 
                 'Hammersmith and Fulham', 'Hampshire', 'Haringey', 'Harrow', 'Hartlepool', 'Havering', 'Herefordshire', 'Hertfordshire', 
                 'Hillingdon', 'Hounslow', 'Isle of Wight', 'Islington', 'Kensington and Chelsea', 'Kent', 'Kingston upon Hull', 
                 'Kingston upon Thames', 'Lambeth', 'Lancashire', 'Leicestershire', 'Leicester', 'Lewisham', 'Lincolnshire', 'London', 
                 'Luton', 'Manchester', 'Medway', 'Merseyside', 'Merton', 'Middlesbrough', 'Milton Keynes', 'Newham', 'Norfolk', 
                 'North East Lincolnshire', 'North Lincolnshire', 'North Somerset', 'North Yorkshire', 'Northamptonshire', 
                 'Northumberland', 'Nottinghamshire', 'Nottingham', 'Oxfordshire', 'Peterborough', 'Plymouth', 'Poole', 'Portsmouth', 
                 'Redbridge', 'Redcar and Cleveland', 'Richmond upon Thames', 'Rutland', 'Shropshire', 'Somerset', 'South Gloucestershire', 
                 'South Yorkshire', 'Southampton', 'Southend-on-Sea', 'Southwark', 'Staffordshire', 'Stockton-on-Tees', 'Stoke-on-Trent', 
                 'Suffolk', 'Surrey', 'Sutton', 'Swindon', 'Telford and Wrekin', 'Thurrock', 'Torbay', 'Tower Hamlets', 'Tyne and Wear', 
                 'Waltham Forest', 'Wandsworth', 'Warrington', 'Warwickshire', 'West Midlands', 'West Sussex', 'West Yorkshire', 
                 'Westminster', 'Wiltshire', 'Worcestershire', 'York', 'Antrim', 'Ards', 'Armagh', 'Ballymena', 'Ballymoney', 'Banbridge', 
                 'Belfast', 'Carrickfergus', 'Castlereagh', 'Coleraine', 'Cookstown', 'Craigavon', 'Derry', 'Down', 'Dungannon', 'Fermanagh', 
                 'Larne', 'Limavady', 'Lisburn', 'Magherafelt', 'Moyle', 'Newry and Mourne', 'Newtownabbey', 'North Down', 'Omagh', 'Strabane', 
                 'Aberdeenshire', 'Aberdeen', 'Angus', 'Argyll and Bute', 'Clackmannanshire', 'Dumfries and Galloway', 'Dundee', 'East Ayrshire', 
                 'East Dunbartonshire', 'East Lothian', 'East Renfrewshire', 'Edinburgh', 'Eilean Siar', 'Falkirk', 'Fife', 'Glasgow', 'Highland', 
                 'Inverclyde', 'Midlothian', 'Moray', 'North Ayshire', 'North Lanarkshire', 'Orkney Islands', 'Perthshire and Kinross', 
                 'Renfrewshire', 'Scottish Borders', 'Shetland Islands', 'South Ayrshire', 'South Lanarkshire', 'Stirling', 'West Dunbartonshire', 
                 'West Lothian', 'Anglesey', 'Blaenau Gwent', 'Bridgend', 'Caerphilly', 'Cardiff', 'Carmarthenshire', 'Ceredigion', 'Conwy', 
                 'Denbighshire', 'Flintshire', 'Gwynedd', 'Merthyr Tydfil', 'Monmouthshire', 'Neath Port Talbot', 'Newport', 'Pembrokeshire', 
                 'Powys', 'Rhondda, Cynon, Taff', 'Swansea', 'Torfaen', 'Vale of Glamorgan', 'Wrexham']

    for region in regions:    
        if region not in stateCounts:
            #print(region)
            stateCounts[region]=0
    print(sorted(regions))                   
    return stateCounts

def avg_FRL(df_state,state):    
    count_retweet=0
    count_follower=0
    count_favorite=0
    count=0
    for index,line in df_state.iterrows():  
        retweet=int(line["Retweets"])
        follower=int(line["Follower"])
        favorites=int(line["Favorites"])

        count_retweet=count_retweet+retweet
        count_follower=count_follower+follower
        count_favorite=count_favorite+favorites
        count=count+1
    calc_follower = int ( count_follower / count)
    calc_retweet = int ( count_retweet / count)
    calc_favorite = int ( count_favorite / count)
    
    return calc_follower, calc_retweet, calc_favorite

# Set Country for Analyses
def analyse_average(country):
    
    regionType ="state"
    
    #For Carmen we need to use column "county"
    #if country =="United Kingdom":
    #    regionType="county"
    
    #Set DataFrame to specific country
    df_country = df_merged[df_merged["country"]==country]

    # Method to match given State Names to State Names from GeoJson
    df_country = edit_states(country,df_country)
    
    #get all states found in dataframe
    states = df_country[regionType].unique()
    states = list(states)
    states = [state for state in states if str(state)!='nan']
    
 
    print(sorted(states))
    
    # Create dict with number of tweets per state
    stateCounts={}
    for state in states:
        stateCounts[state]=len(df_country[df_country[regionType]==state])

    #Get average per State:
    stateFollower = {}
    stateRetweets = {}
    stateLikes = {}
    
    #States without tweets get value 0
    data = fill_states(country,stateCounts)
    
    # Fill dict ->  statename : average Follower    
    for state in data:
        if state not in states:
            stateFollower[state]=0
            stateRetweets[state] = 0
            stateLikes[state] = 0
            continue
            
        df_state = df_country[df_country[regionType]==state]
        follower,retweet,like = avg_FRL(df_state,state)
        stateFollower[state]= follower
        stateRetweets[state] = retweet
        stateLikes[state] = like
    
    return stateFollower, stateRetweets, stateLikes

In [ ]:
germany_Follower, germany_Retweets, germany_Likes = analyse_average("Germany")

In [ ]:
#Germany average Follower

label = Label()
data = germany_Follower

colorScheme = ["#FFFCFC", "#F1DCDC","#DB8585", "#BB1216","#6c0000"]
colormap_dept = color.StepColormap(colors=colorScheme,vmin=min(data.values()),vmax=max(data.values()),index=[0,1000, 10000, 100000,500000])
legend = LegendControl({"0 - 1K":colorScheme[0], "1K - 10K":colorScheme[1], "10K - 100K":colorScheme[2],"100K - 500K":colorScheme[3],"500K+": colorScheme[4]},fontsize=2, name="Avg. Follower", position="bottomleft")

m = Map(layers=(),center=(51.2252978589571, 9.84580993652344),zoom_control=False,zoom=6)

layer = ipyleaflet.Choropleth(geo_data= german_data,choro_data=germany_Follower,key_on="id",colormap=colormap_dept,style={'fillOpacity': 1}
)

layer.on_click(set_popUp)
m.on_interaction(handle_interaction_country)
m.add_layer(layer)
m.add_control(legend)
m.layout.width = '60%'
m.layout.height = '600px'
m

In [43]:
#Germany average retweets
label = Label()
data = germany_Retweets
colorScheme = ["#FFFCFC", "#F1DCDC","#DB8585", "#BB1216","#6c0000"]
colormap_dept = color.StepColormap(colors=colorScheme,vmin=min(data.values()),vmax=max(data.values()),index=[0,10, 25, 50,100])
legend = LegendControl({"0 - 10":colorScheme[0], "10 - 25":colorScheme[1],"25 - 50":colorScheme[2],"50 - 75":colorScheme[3],"100+": colorScheme[4]},fontsize=2, name="Avg. Retweets", position="bottomleft")

m = Map(layers=(),center=(51.2252978589571, 9.84580993652344),zoom_control=False,zoom=6)

layer = ipyleaflet.Choropleth(geo_data= german_data,choro_data=data,key_on="id",colormap=colormap_dept,style={'fillOpacity': 1}
)

layer.on_click(set_popUp)
m.on_interaction(handle_interaction_country)
m.add_layer(layer)
m.add_control(legend)
m.layout.width = '60%'
m.layout.height = '600px'
m

Map(center=[51.2252978589571, 9.84580993652344], controls=(AttributionControl(options=['position', 'prefix'], …

In [44]:
#Germany average likes
label = Label()
data = germany_Likes
colorScheme = ["#FFFCFC", "#F1DCDC","#DB8585", "#BB1216","#6c0000"]
colormap_dept = color.StepColormap(colors=colorScheme,vmin=min(data.values()),vmax=max(data.values()),index=[0,25, 50, 100,250])
legend = LegendControl({"0 - 25":colorScheme[0], "25 - 50":colorScheme[1],"50 - 100":colorScheme[2],"100 - 250":colorScheme[3],"250+": colorScheme[4]},fontsize=2, name="Avg. Likes", position="bottomleft")

m = Map(layers=(),center=(51.2252978589571, 9.84580993652344),zoom_control=False,zoom=6)

layer = ipyleaflet.Choropleth(geo_data= german_data,choro_data=data,key_on="id",colormap=colormap_dept,style={'fillOpacity': 1}
)

layer.on_click(set_popUp)
m.on_interaction(handle_interaction_country)
m.add_layer(layer)
m.add_control(legend)
m.layout.width = '60%'
m.layout.height = '600px'
m

Map(center=[51.2252978589571, 9.84580993652344], controls=(AttributionControl(options=['position', 'prefix'], …

In [45]:
spain_Follower, spain_Retweets, spain_Likes = analyse_average("Spain")

['Andalucia', 'Aragon', 'Asturias', 'Baleares', 'Canarias', 'Cantabria', 'Castilla-La Mancha', 'Castilla-Leon', 'Catalonia', 'Extremadura', 'Galicia', 'La Rioja', 'Madrid', 'Melilla', 'Murcia', 'Navarra', 'Pais Vasco', 'Valencia']
['Andalucia', 'Aragon', 'Asturias', 'Baleares', 'Canarias', 'Cantabria', 'Castilla-La Mancha', 'Castilla-Leon', 'Catalonia', 'Ceuta', 'Extremadura', 'Galicia', 'La Rioja', 'Madrid', 'Melilla', 'Murcia', 'Navarra', 'Pais Vasco', 'Valencia']


In [46]:
#Spain average Follower

label = Label()
data = spain_Follower
colorScheme = ["#FFFCFC", "#F1DCDC","#DB8585", "#BB1216","#6c0000"]
colormap_dept = color.StepColormap(colors=colorScheme,vmin=min(data.values()),vmax=max(data.values()),index=[0,1000, 10000, 100000,500000])
legend = LegendControl({"0 - 1K":colorScheme[0], "1K - 10K":colorScheme[1], "10K - 100K":colorScheme[2],"100K - 500K":colorScheme[3],"500K+": colorScheme[4]},fontsize=2, name="Avg. Follower", position="bottomright")

m = Map(layers=(), center=( 39.8252978589571, -2.54580993652344),zoom_control=False, zoom=6)

layer = ipyleaflet.Choropleth(geo_data= spain_data,choro_data=data,key_on="id",colormap=colormap_dept,style={'fillOpacity': 1}
)

layer.on_click(set_popUp)
m.on_interaction(handle_interaction_country)
m.add_layer(layer)
m.add_control(legend)
m.layout.width = '65%'
m.layout.height = '520px'
m

Map(center=[39.8252978589571, -2.54580993652344], controls=(AttributionControl(options=['position', 'prefix'],…

In [47]:
#Spain average retweets
label = Label()
data = spain_Retweets

colorScheme = ["#FFFCFC", "#F1DCDC","#DB8585", "#BB1216","#6c0000"]
colormap_dept = color.StepColormap(colors=colorScheme,vmin=min(data.values()),vmax=max(data.values()),index=[0,10, 25, 50,100])
legend = LegendControl({"0 - 10":colorScheme[0], "10 - 25":colorScheme[1],"25 - 50":colorScheme[2],"50 - 75":colorScheme[3],"100+": colorScheme[4]},fontsize=2, name="Avg. Retweets", position="bottomright")

m = Map(layers=(), center=( 39.8252978589571, -2.54580993652344),zoom_control=False, zoom=6)

layer = ipyleaflet.Choropleth(geo_data= spain_data,choro_data=data,key_on="id",colormap=colormap_dept,style={'fillOpacity': 1}
)

layer.on_click(set_popUp)
m.on_interaction(handle_interaction_country)
m.add_layer(layer)
m.add_control(legend)
m.layout.width = '65%'
m.layout.height = '520px'
m

Map(center=[39.8252978589571, -2.54580993652344], controls=(AttributionControl(options=['position', 'prefix'],…

In [48]:
#Spain average likes
label = Label()
data = spain_Likes
colorScheme = ["#FFFCFC", "#F1DCDC","#DB8585", "#BB1216","#6c0000"]
colormap_dept = color.StepColormap(colors=colorScheme,vmin=min(data.values()),vmax=max(data.values()),index=[0,25, 50, 100,250])
legend = LegendControl({"0 - 25":colorScheme[0], "25 - 50":colorScheme[1],"50 - 100":colorScheme[2],"100 - 250":colorScheme[3],"250+": colorScheme[4]},fontsize=2, name="Avg. Likes", position="bottomright")

m = Map(layers=(), center=( 39.8252978589571, -2.54580993652344),zoom_control=False, zoom=6)

layer = ipyleaflet.Choropleth(geo_data= spain_data,choro_data=data,key_on="id",colormap=colormap_dept,style={'fillOpacity': 1}
)

layer.on_click(set_popUp)
m.on_interaction(handle_interaction_country)
m.add_layer(layer)
m.add_control(legend)
m.layout.width = '65%'
m.layout.height = '520px'
m

Map(center=[39.8252978589571, -2.54580993652344], controls=(AttributionControl(options=['position', 'prefix'],…

In [49]:
france_Follower, france_Retweets, france_Likes = analyse_average("France")

['Auvergne-Rhône-Alpes', 'Bourgogne-Franche-Comté', 'Bretagne', 'Centre-Val de Loire', 'Corse', 'Grand Est', 'Hauts-de-France', 'Normandie', 'Nouvelle-Aquitaine', 'Occitanie', 'Pays de la Loire', "Provence-Alpes-Côte d'Azur", 'Île-de-France']
['Auvergne-Rhône-Alpes', 'Bourgogne-Franche-Comté', 'Bretagne', 'Centre-Val de Loire', 'Corse', 'Grand Est', 'Guadeloupe', 'Guyane', 'Hauts-de-France', 'La Réunion', 'Martinique', 'Mayotte', 'Normandie', 'Nouvelle-Aquitaine', 'Occitanie', 'Pays de la Loire', "Provence-Alpes-Côte d'Azur", 'Île-de-France']


In [50]:
#France average Follower

label = Label()
data = france_Follower

colorScheme = ["#FFFCFC", "#F1DCDC","#DB8585", "#BB1216","#6c0000"]
colormap_dept = color.StepColormap(colors=colorScheme,vmin=min(data.values()),vmax=max(data.values()),index=[0,1000, 10000, 100000,500000])
legend = LegendControl({"0 - 1K":colorScheme[0], "1K - 10K":colorScheme[1], "10K - 100K":colorScheme[2],"100K - 500K":colorScheme[3],"500K+": colorScheme[4]},fontsize=2, name="Avg. Follower", position="bottomleft")

m = Map(layers=(), center=( 46.42252978589571, 2.99580993652344),zoom_control=False ,zoom=6)
layer = ipyleaflet.Choropleth(geo_data= france_data,choro_data=data,key_on="id",colormap=colormap_dept,style={'fillOpacity': 1}
)

layer.on_click(set_popUp)
m.on_interaction(handle_interaction_country)
m.add_layer(layer)
m.add_control(legend)
m.layout.width = '72%'
m.layout.height = '650px'
m

Map(center=[46.42252978589571, 2.99580993652344], controls=(AttributionControl(options=['position', 'prefix'],…

In [51]:
#France average retweets
label = Label()
data = france_Retweets

colorScheme = ["#FFFCFC", "#F1DCDC","#DB8585", "#BB1216","#6c0000"]
colormap_dept = color.StepColormap(colors=colorScheme,vmin=min(data.values()),vmax=max(data.values()),index=[0,10, 25, 50,100])
legend = LegendControl({"0 - 10":colorScheme[0], "10 - 25":colorScheme[1],"25 - 50":colorScheme[2],"50 - 75":colorScheme[3],"100+": colorScheme[4]},fontsize=2, name="Avg. Retweets", position="bottomleft")

m = Map(layers=(), center=( 46.42252978589571, 2.99580993652344),zoom_control=False ,zoom=6)

layer = ipyleaflet.Choropleth(geo_data= france_data,choro_data=data,key_on="id",colormap=colormap_dept,style={'fillOpacity': 1}
)

layer.on_click(set_popUp)
m.on_interaction(handle_interaction_country)
m.add_layer(layer)
m.add_control(legend)
m.layout.width = '72%'
m.layout.height = '650px'
m

Map(center=[46.42252978589571, 2.99580993652344], controls=(AttributionControl(options=['position', 'prefix'],…

In [52]:
#France average likes
label = Label()
data = france_Likes
colorScheme = ["#FFFCFC", "#F1DCDC","#DB8585", "#BB1216","#6c0000"]
colormap_dept = color.StepColormap(colors=colorScheme,vmin=min(data.values()),vmax=max(data.values()),index=[0,25, 50, 100,250])
legend = LegendControl({"0 - 25":colorScheme[0], "25 - 50":colorScheme[1],"50 - 100":colorScheme[2],"100 - 250":colorScheme[3],"250+": colorScheme[4]},fontsize=2, name="Avg. Likes", position="bottomleft")

m = Map(layers=(), center=( 46.42252978589571, 2.99580993652344),zoom_control=False ,zoom=6)

layer = ipyleaflet.Choropleth(geo_data= france_data,choro_data=data,key_on="id",colormap=colormap_dept,style={'fillOpacity': 1}
)

layer.on_click(set_popUp)
m.on_interaction(handle_interaction_country)
m.add_layer(layer)
m.add_control(legend)
m.layout.width = '72%'
m.layout.height = '650px'
m

Map(center=[46.42252978589571, 2.99580993652344], controls=(AttributionControl(options=['position', 'prefix'],…

In [56]:
italy_Follower, italy_Retweets, italy_Likes = analyse_average("Italy")

['Abruzzo', 'Basilicata', 'Calabria', 'Campania', 'Emilia-Romagna', 'Friuli-Venezia Giulia', 'Lazio', 'Liguria', 'Lombardia', 'Marche', 'Piemonte', 'Puglia', 'Sardegna', 'Sicilia', 'Toscana', 'Trentino-Alto Adige', 'Umbria', "Valle d'Aosta", 'Veneto']
['Abruzzo', 'Basilicata', 'Calabria', 'Campania', 'Emilia-Romagna', 'Friuli-Venezia Giulia', 'Lazio', 'Liguria', 'Lombardia', 'Marche', 'Molise', 'Piemonte', 'Puglia', 'Sardegna', 'Sicilia', 'Toscana', 'Trentino-Alto Adige', 'Umbria', "Valle d'Aosta", 'Veneto']


In [57]:
#Italy average Follower

label = Label()
data = italy_Follower
colorScheme = ["#FFFCFC", "#F1DCDC","#DB8585", "#BB1216","#6c0000"]
colormap_dept = color.StepColormap(colors=colorScheme,vmin=min(data.values()),vmax=max(data.values()),index=[0,1000, 10000, 100000,500000])
legend = LegendControl({"0 - 1K":colorScheme[0], "1K - 10K":colorScheme[1], "10K - 100K":colorScheme[2],"100K - 500K":colorScheme[3],"500K+": colorScheme[4]},fontsize=2, name="Avg. Follower", position="bottomleft")

m = Map(layers=(), center=( 41.9952978589571, 12.90),zoom_control=False, zoom=6)


layer = ipyleaflet.Choropleth(geo_data= italy_data,choro_data=data,key_on="id",colormap=colormap_dept,style={'fillOpacity': 1}
)

layer.on_click(set_popUp)
m.on_interaction(handle_interaction_country)
m.add_layer(layer)
m.add_control(legend)
m.layout.width = '63%'
m.layout.height = '660px'
m

Map(center=[41.9952978589571, 12.9], controls=(AttributionControl(options=['position', 'prefix'], position='bo…

In [58]:
#Italy average retweets
label = Label()
data = italy_Retweets
colorScheme = ["#FFFCFC", "#F1DCDC","#DB8585", "#BB1216","#6c0000"]
colormap_dept = color.StepColormap(colors=colorScheme,vmin=min(data.values()),vmax=max(data.values()),index=[0,10, 25, 50,100])
legend = LegendControl({"0 - 10":colorScheme[0], "10 - 25":colorScheme[1],"25 - 50":colorScheme[2],"50 - 75":colorScheme[3],"100+": colorScheme[4]},fontsize=2, name="Avg. Retweets", position="bottomleft")

m = Map(layers=(), center=( 41.9952978589571, 12.90),zoom_control=False, zoom=6)
layer = ipyleaflet.Choropleth(geo_data= italy_data,choro_data=data,key_on="id",colormap=colormap_dept,style={'fillOpacity': 1}
)

layer.on_click(set_popUp)
m.on_interaction(handle_interaction_country)
m.add_layer(layer)
m.add_control(legend)
m.layout.width = '63%'
m.layout.height = '660px'
m

Map(center=[41.9952978589571, 12.9], controls=(AttributionControl(options=['position', 'prefix'], position='bo…

In [59]:
#Italy average likes
label = Label()
data = italy_Likes
colorScheme = ["#FFFCFC", "#F1DCDC","#DB8585", "#BB1216","#6c0000"]
colormap_dept = color.StepColormap(colors=colorScheme,vmin=min(data.values()),vmax=max(data.values()),index=[0,25, 50, 100,250])
legend = LegendControl({"0 - 25":colorScheme[0], "25 - 50":colorScheme[1],"50 - 100":colorScheme[2],"100 - 250":colorScheme[3],"250+": colorScheme[4]},fontsize=2, name="Avg. Likes", position="bottomleft")

m = Map(layers=(), center=( 41.9952978589571, 12.90),zoom_control=False, zoom=6)

layer = ipyleaflet.Choropleth(geo_data= italy_data,choro_data=data,key_on="id",colormap=colormap_dept,style={'fillOpacity': 1}
)

layer.on_click(set_popUp)
m.on_interaction(handle_interaction_country)
m.add_layer(layer)
m.add_control(legend)
m.layout.width = '63%'
m.layout.height = '660px'
m

Map(center=[41.9952978589571, 12.9], controls=(AttributionControl(options=['position', 'prefix'], position='bo…

In [60]:
india_Follower, india_Retweets, india_Likes = analyse_average("India")

['Andaman & Nicobar Island', 'Andhra Pradesh', 'Assam', 'Bihar', 'Chandigarh', 'Chhattisgarh', 'Goa', 'Gujarat', 'Haryana', 'Himachal Pradesh', 'Jammu & Kashmir', 'Jharkhand', 'Karnataka', 'Kerala', 'Laccadives', 'Madhya Pradesh', 'Maharashtra', 'Meghalaya', 'Mizoram', 'NCT of Delhi', 'Odisha', 'Punjab', 'Rajasthan', 'Tamil Nadu', 'Telangana', 'Tripura', 'Uttar Pradesh', 'Uttarakhand', 'West Bengal']
['Andaman & Nicobar Island', 'Andhra Pradesh', 'Arunanchal Pradesh', 'Assam', 'Bihar', 'Chandigarh', 'Chhattisgarh', 'Dadara & Nagar Havelli', 'Daman & Diu', 'Goa', 'Gujarat', 'Haryana', 'Himachal Pradesh', 'Jammu & Kashmir', 'Jharkhand', 'Karnataka', 'Kerala', 'Lakshadweep', 'Madhya Pradesh', 'Maharashtra', 'Manipur', 'Meghalaya', 'Mizoram', 'NCT of Delhi', 'Nagaland', 'Odisha', 'Puducherry', 'Punjab', 'Rajasthan', 'Sikkim', 'Tamil Nadu', 'Telangana', 'Tripura', 'Uttar Pradesh', 'Uttarakhand', 'West Bengal']


In [61]:
#India average Follower

label = Label()
data = india_Follower
colorScheme = ["#FFFCFC", "#F1DCDC","#DB8585", "#BB1216","#6c0000"]
colormap_dept = color.StepColormap(colors=colorScheme,vmin=min(data.values()),vmax=max(data.values()),index=[0,1000, 10000, 100000,500000])
legend = LegendControl({"0 - 1K":colorScheme[0], "1K - 10K":colorScheme[1], "10K - 100K":colorScheme[2],"100K - 500K":colorScheme[3],"500K+": colorScheme[4]},fontsize=2, name="Avg. Follower", position="bottomleft")

m = Map(layers=(), center=( 23.2252978589571, 84.44580993652344), zoom_control=False,zoom=5)

layer = ipyleaflet.Choropleth(geo_data= india_data,choro_data=data,key_on="id",colormap=colormap_dept,style={'fillOpacity': 1}
)

layer.on_click(set_popUp)
m.on_interaction(handle_interaction_country)
m.add_layer(layer)
m.add_control(legend)
m.layout.width = '76%'
m.layout.height = '750px'
m

Map(center=[23.2252978589571, 84.44580993652345], controls=(AttributionControl(options=['position', 'prefix'],…

In [62]:
#India average retweets
label = Label()
data = india_Retweets
colorScheme = ["#FFFCFC", "#F1DCDC","#DB8585", "#BB1216","#6c0000"]
colormap_dept = color.StepColormap(colors=colorScheme,vmin=min(data.values()),vmax=max(data.values()),index=[0,10, 25, 50,100])
legend = LegendControl({"0 - 10":colorScheme[0], "10 - 25":colorScheme[1],"25 - 50":colorScheme[2],"50 - 75":colorScheme[3],"100+": colorScheme[4]},fontsize=2, name="Avg. Retweets", position="bottomleft")

m = Map(layers=(), center=( 23.2252978589571, 84.44580993652344), zoom_control=False,zoom=5)

layer = ipyleaflet.Choropleth(geo_data= india_data,choro_data=data,key_on="id",colormap=colormap_dept,style={'fillOpacity': 1}
)

layer.on_click(set_popUp)
m.on_interaction(handle_interaction_country)
m.add_layer(layer)
m.add_control(legend)
m.layout.width = '76%'
m.layout.height = '750px'
m

Map(center=[23.2252978589571, 84.44580993652345], controls=(AttributionControl(options=['position', 'prefix'],…

In [63]:
#India average likes
label = Label()
data = india_Likes
colorScheme = ["#FFFCFC", "#F1DCDC","#DB8585", "#BB1216","#6c0000"]
colormap_dept = color.StepColormap(colors=colorScheme,vmin=min(data.values()),vmax=max(data.values()),index=[0,25, 50, 100,250])
legend = LegendControl({"0 - 25":colorScheme[0], "25 - 50":colorScheme[1],"50 - 100":colorScheme[2],"100 - 250":colorScheme[3],"250+": colorScheme[4]},fontsize=2, name="Avg. Likes", position="bottomleft")

m = Map(layers=(), center=( 23.2252978589571, 84.44580993652344), zoom_control=False,zoom=5)

layer = ipyleaflet.Choropleth(geo_data= india_data,choro_data=data,key_on="id",colormap=colormap_dept,style={'fillOpacity': 1}
)

layer.on_click(set_popUp)
m.on_interaction(handle_interaction_country)
m.add_layer(layer)
m.add_control(legend)
m.layout.width = '76%'
m.layout.height = '750px'
m

Map(center=[23.2252978589571, 84.44580993652345], controls=(AttributionControl(options=['position', 'prefix'],…

In [64]:
us_Follower, us_Retweets, us_Likes = analyse_average("United States")

['Alabama', 'Alaska', 'Arizona', 'Arkansas', 'California', 'Colorado', 'Connecticut', 'Delaware', 'District of Columbia', 'Florida', 'Georgia', 'Hawaii', 'Idaho', 'Illinois', 'Indiana', 'Iowa', 'Kansas', 'Kentucky', 'Louisiana', 'Maine', 'Maryland', 'Massachusetts', 'Michigan', 'Minnesota', 'Mississippi', 'Missouri', 'Montana', 'Nebraska', 'Nevada', 'New Hampshire', 'New Jersey', 'New Mexico', 'New York', 'North Carolina', 'North Dakota', 'Ohio', 'Oklahoma', 'Oregon', 'Pennsylvania', 'Rhode Island', 'South Carolina', 'South Dakota', 'Tennessee', 'Texas', 'Utah', 'Vermont', 'Virginia', 'Washington', 'West Virginia', 'Wisconsin', 'Wyoming']
['Alabama', 'Alaska', 'Arizona', 'Arkansas', 'California', 'Colorado', 'Connecticut', 'Delaware', 'District of Columbia', 'Florida', 'Georgia', 'Hawaii', 'Idaho', 'Illinois', 'Indiana', 'Iowa', 'Kansas', 'Kentucky', 'Louisiana', 'Maine', 'Maryland', 'Massachusetts', 'Michigan', 'Minnesota', 'Mississippi', 'Missouri', 'Montana', 'Nebraska', 'Nevada', '

In [65]:
#United States average Follower

label = Label()
data = us_Follower
colorScheme = ["#FFFCFC", "#F1DCDC","#DB8585", "#BB1216","#6c0000"]
colormap_dept = color.StepColormap(colors=colorScheme,vmin=min(data.values()),vmax=max(data.values()),index=[0,1000, 10000, 100000,500000])
legend = LegendControl({"0 - 1K":colorScheme[0], "1K - 10K":colorScheme[1], "10K - 100K":colorScheme[2],"100K - 500K":colorScheme[3],"500K+": colorScheme[4]},fontsize=2, name="Avg. Follower", position="bottomleft")

m = Map(layers=(), center=( 37.6252978589571, -94.24580993652344),zoom_control=False,zoom=4)
layer = ipyleaflet.Choropleth(geo_data= us_data,choro_data=data,key_on="id",colormap=colormap_dept,style={'fillOpacity': 1}
)

layer.on_click(set_popUp)
m.on_interaction(handle_interaction_country)
m.add_layer(layer)
m.add_control(legend)
m.layout.width = '75%'
m.layout.height = '440px'
m

Map(center=[37.6252978589571, -94.24580993652344], controls=(AttributionControl(options=['position', 'prefix']…

In [66]:
#United States retweets
label = Label()
data = us_Retweets
colorScheme = ["#FFFCFC", "#F1DCDC","#DB8585", "#BB1216","#6c0000"]
colormap_dept = color.StepColormap(colors=colorScheme,vmin=min(data.values()),vmax=max(data.values()),index=[0,10, 25, 50,100])
legend = LegendControl({"0 - 10":colorScheme[0], "10 - 25":colorScheme[1],"25 - 50":colorScheme[2],"50 - 75":colorScheme[3],"100+": colorScheme[4]},fontsize=2, name="Avg. Retweets", position="bottomleft")

m = Map(layers=(), center=( 37.6252978589571, -94.24580993652344),zoom_control=False,zoom=4)

layer = ipyleaflet.Choropleth(geo_data= us_data,choro_data=data,key_on="id",colormap=colormap_dept,style={'fillOpacity': 1}
)

layer.on_click(set_popUp)
m.on_interaction(handle_interaction_country)
m.add_layer(layer)
m.add_control(legend)
m.layout.width = '75%'
m.layout.height = '440px'
m

Map(center=[37.6252978589571, -94.24580993652344], controls=(AttributionControl(options=['position', 'prefix']…

In [67]:
#United States average likes
label = Label()
data = us_Likes
colorScheme = ["#FFFCFC", "#F1DCDC","#DB8585", "#BB1216","#6c0000"]
colormap_dept = color.StepColormap(colors=colorScheme,vmin=min(data.values()),vmax=max(data.values()),index=[0,25, 50, 100,250])
legend = LegendControl({"0 - 25":colorScheme[0], "25 - 50":colorScheme[1],"50 - 100":colorScheme[2],"100 - 250":colorScheme[3],"250+": colorScheme[4]},fontsize=2, name="Avg. Likes", position="bottomleft")

m = Map(layers=(), center=( 37.6252978589571, -94.24580993652344),zoom_control=False,zoom=4)

layer = ipyleaflet.Choropleth(geo_data= us_data,choro_data=data,key_on="id",colormap=colormap_dept,style={'fillOpacity': 1}
)

layer.on_click(set_popUp)
m.on_interaction(handle_interaction_country)
m.add_layer(layer)
m.add_control(legend)
m.layout.width = '75%'
m.layout.height = '440px'
m

Map(center=[37.6252978589571, -94.24580993652344], controls=(AttributionControl(options=['position', 'prefix']…

In [68]:
uk_Follower, uk_Retweets, uk_Likes = analyse_average("United Kingdom")

['Aberdeen', 'Aberdeenshire', 'Anglesey', 'Bath and North East Somerset', 'Bedfordshire', 'Belfast', 'Berkshire', 'Blackburn with Darwen', 'Blaenau Gwent', 'Bournemouth', 'Bridgend', 'Brighton and Hove', 'Bristol', 'Buckinghamshire', 'Caerphilly', 'Cambridgeshire', 'Cardiff', 'Carmarthenshire', 'Ceredigion', 'Cheshire', 'Chesire', 'Conwy', 'Cornwall', 'Cumbria', 'Darlington', 'Denbighshire', 'Derby', 'Derbyshire', 'Devon', 'Dorset', 'Dumfries and Galloway', 'Dundee', 'Durham', 'East Lothian', 'East Renfrewshire', 'East Riding of Yorkshire', 'East Sussex', 'Edinburgh', 'Essex', 'Falkirk', 'Fife', 'Flintshire', 'Glasgow', 'Gloucestershire', 'Gwynedd', 'Hampshire', 'Herefordshire', 'Hertfordshire', 'Isle of Wight', 'Isles of Scilly', 'Kent', 'Lancashire', 'Leicestershire', 'Lincolnshire', 'London', 'Luton', 'Manchester', 'Medway', 'Merseyside', 'Middlesbrough', 'Midlothian', 'Milton Keynes', 'Monmouthshire', 'Newport', 'Norfolk', 'North East Lincolnshire', 'North Lanarkshire', 'North Linc

In [69]:
#United Kingdom average Follower

label = Label()
data = uk_Follower
colorScheme = ["#FFFCFC", "#F1DCDC","#DB8585", "#BB1216","#6c0000"]
colormap_dept = color.StepColormap(colors=colorScheme,vmin=min(data.values()),vmax=max(data.values()),index=[0,1000, 10000, 100000,500000])
legend = LegendControl({"0 - 1K":colorScheme[0], "1K - 10K":colorScheme[1], "10K - 100K":colorScheme[2],"100K - 500K":colorScheme[3],"500K+": colorScheme[4]},fontsize=2, name="Avg. Follower", position="bottomleft")

m = Map(layers=(), center=( 54.9, -2.74580993652344),zoom_control=False,zoom=6)
layer = ipyleaflet.Choropleth(geo_data= uk_data,choro_data=data,key_on="id",colormap=colormap_dept,style={'fillOpacity': 1}
)

layer.on_click(set_popUp)
m.on_interaction(handle_interaction_country)
m.add_layer(layer)
m.add_control(legend)
m.layout.width = '60%'
m.layout.height = '750px'
m

Map(center=[54.9, -2.74580993652344], controls=(AttributionControl(options=['position', 'prefix'], position='b…

In [70]:
#United Kingdom average Retweets

label = Label()
data = uk_Retweets
colorScheme = ["#FFFCFC", "#F1DCDC","#DB8585", "#BB1216","#6c0000"]
colormap_dept = color.StepColormap(colors=colorScheme,vmin=min(data.values()),vmax=max(data.values()),index=[0,10, 25, 50,100])
legend = LegendControl({"0 - 10":colorScheme[0], "10 - 25":colorScheme[1],"25 - 50":colorScheme[2],"50 - 75":colorScheme[3],"100+": colorScheme[4]},fontsize=2, name="Avg. Retweets", position="bottomleft")

m = Map(layers=(), center=( 54.9, -2.74580993652344),zoom_control=False,zoom=6)
layer = ipyleaflet.Choropleth(geo_data= uk_data,choro_data=data,key_on="id",colormap=colormap_dept,style={'fillOpacity': 1}
)

layer.on_click(set_popUp)
m.on_interaction(handle_interaction_country)
m.add_layer(layer)
m.add_control(legend)
m.layout.width = '60%'
m.layout.height = '750px'
m

Map(center=[54.9, -2.74580993652344], controls=(AttributionControl(options=['position', 'prefix'], position='b…

In [71]:
#United Kingdom average Likes

label = Label()
data = uk_Likes
colorScheme = ["#FFFCFC", "#F1DCDC","#DB8585", "#BB1216","#6c0000"]
colormap_dept = color.StepColormap(colors=colorScheme,vmin=min(data.values()),vmax=max(data.values()),index=[0,25, 50, 100,250])
legend = LegendControl({"0 - 25":colorScheme[0], "25 - 50":colorScheme[1],"50 - 100":colorScheme[2],"100 - 250":colorScheme[3],"250+": colorScheme[4]},fontsize=2, name="Avg. Likes", position="bottomleft")

m = Map(layers=(), center=( 54.9, -2.74580993652344),zoom_control=False,zoom=6)
layer = ipyleaflet.Choropleth(geo_data= uk_data,choro_data=data,key_on="id",colormap=colormap_dept,style={'fillOpacity': 1}
)

layer.on_click(set_popUp)
m.on_interaction(handle_interaction_country)
m.add_layer(layer)
m.add_control(legend)
m.layout.width = '60%'
m.layout.height = '750px'
m

Map(center=[54.9, -2.74580993652344], controls=(AttributionControl(options=['position', 'prefix'], position='b…

In [72]:
#************************************************************************************************

In [73]:
#************************************************************************************************

In [74]:
# Analyse Tweets count per Country

In [75]:
def analyse_tweetsCount(country):
    regionType ="state"   
    
    #For Carmen set regionType to County
    #if country =="United Kingdom":
    #    regionType="county"
        
    #Set DataFrame to specific country
    df_country = df_merged[df_merged["country"]==country]

    # Method to match given State Names to State Names from GeoJson
    df_country = edit_states(country,df_country)
    
    #get all states found in dataframe
    states = df_country[regionType].unique()

    #Tweets for specific country without state information
    statesNotFound =len(df_country[df_country[regionType]=="null"])
    
    #Get Tweet Count per State:
    stateCounts={}
    
    # Fill dict ->  statename : countTweets
    for state in states:
        stateCounts[state]=len(df_country[df_country[regionType]==state])
       
    #States without tweets get value 0
    data = fill_states(country,stateCounts)
    
    return data, statesNotFound

In [76]:
from ipyleaflet import Map, basemaps
data, countNull = analyse_tweetsCount("Germany")
print(data)
label = Label()

colorScheme = ["#FFFCFC", "#F1DCDC","#DB8585", "#BB1216","#6c0000"]
colormap_dept = color.StepColormap(colors=colorScheme,vmin=min(data.values()),vmax=max(data.values()),index=[0, 100, 1000, 10000, 50000,100000])
legend = LegendControl({"0 - 100":colorScheme[0], "100 - 1K":colorScheme[1],"1K - 10K":colorScheme[2],"10K - 50K":colorScheme[3],"50K+": colorScheme[4]},fontsize=2, name="Tweets count", position="bottomleft")

m = Map(layers=(),center=(51.2252978589571, 9.84580993652344),zoom_control=False,zoom=6)

layer = ipyleaflet.Choropleth(
    geo_data= german_data,
    choro_data=data,
    key_on="id",
    colormap=colormap_dept,
    style={'fillOpacity': 1}
)

layer.on_click(set_popUp)
m.on_interaction(handle_interaction_country)
m.add_layer(layer)
m.add_control(legend)
m.layout.width = '60%'
m.layout.height = '600px'
m


['Baden-Württemberg', 'Bayern', 'Berlin', 'Brandenburg', 'Bremen', 'Hamburg', 'Hessen', 'Mecklenburg-Vorpommern', 'Niedersachsen', 'Nordrhein-Westfalen', 'Rheinland-Pfalz', 'Saarland', 'Sachsen', 'Sachsen-Anhalt', 'Schleswig-Holstein', 'Thüringen']
{'Berlin': 632, 'Saarland': 120, 'Hessen': 284, 'Sachsen': 162, 'Nordrhein-Westfalen': 623, 'Niedersachsen': 160, 'Bayern': 506, 'Sachsen-Anhalt': 36, 'Schleswig-Holstein': 46, 'Thüringen': 47, 'Baden-Württemberg': 401, 'Mecklenburg-Vorpommern': 3, 'Rheinland-Pfalz': 85, 'Brandenburg': 26, 'Hamburg': 60, 'Bremen': 1}


Map(center=[51.2252978589571, 9.84580993652344], controls=(AttributionControl(options=['position', 'prefix'], …

In [77]:
data, countNull = analyse_tweetsCount("Spain")
label = Label()

colorScheme = ["#FFFCFC", "#F1DCDC","#DB8585", "#BB1216","#6c0000"]
colormap_dept = color.StepColormap(colors=colorScheme,vmin=min(data.values()),vmax=max(data.values()),index=[0, 100, 1000, 10000, 50000,100000])
legend = LegendControl({"0 - 100":colorScheme[0], "100 - 1K":colorScheme[1],"1K - 10K":colorScheme[2],"10K - 50K":colorScheme[3],"50K+": colorScheme[4]},fontsize=2, name="Tweets count", position="bottomright")

m = Map(layers=(), center=( 39.8252978589571, -2.54580993652344),zoom_control=False, zoom=6)

layer = ipyleaflet.Choropleth(
    geo_data=spain_data,
    choro_data=data,
    colormap=colormap_dept,
    style={'fillOpacity': 1}
)

layer.on_click(set_popUp)
m.on_interaction(handle_interaction_country)
m.add_layer(layer)
m.add_control(legend)
m.layout.width = '65%'
m.layout.height = '520px'
m


['Andalucia', 'Aragon', 'Asturias', 'Baleares', 'Canarias', 'Cantabria', 'Castilla-La Mancha', 'Castilla-Leon', 'Catalonia', 'Ceuta', 'Extremadura', 'Galicia', 'La Rioja', 'Madrid', 'Melilla', 'Murcia', 'Navarra', 'Pais Vasco', 'Valencia']


Map(center=[39.8252978589571, -2.54580993652344], controls=(AttributionControl(options=['position', 'prefix'],…

In [78]:
data, countNull = analyse_tweetsCount("France")
print(data)
label = Label()

colorScheme = ["#FFFCFC", "#F1DCDC","#DB8585", "#BB1216","#6c0000"]
colormap_dept = color.StepColormap(colors=colorScheme,vmin=min(data.values()),vmax=max(data.values()),index=[0, 100, 1000, 10000, 50000,100000])
legend = LegendControl({"0 - 100":colorScheme[0], "100 - 1K":colorScheme[1],"1K - 10K":colorScheme[2],"10K - 50K":colorScheme[3],"50K+": colorScheme[4]},fontsize=2, name="Tweets count", position="bottomleft")

m = Map(layers=(), center=( 46.42252978589571, 2.99580993652344),zoom_control=False ,zoom=6)
layer = ipyleaflet.Choropleth(
    geo_data=france_data,
    choro_data=data,
    key_on='id',
    colormap=colormap_dept,
    style={'fillOpacity': 0.9}
)

layer.on_click(my_method)
m.on_interaction(handle_interaction)
m.add_layer(layer)
m.add_control(legend)
m.layout.width = '72%'
m.layout.height = '650px'
m


['Auvergne-Rhône-Alpes', 'Bourgogne-Franche-Comté', 'Bretagne', 'Centre-Val de Loire', 'Corse', 'Grand Est', 'Guadeloupe', 'Guyane', 'Hauts-de-France', 'La Réunion', 'Martinique', 'Mayotte', 'Normandie', 'Nouvelle-Aquitaine', 'Occitanie', 'Pays de la Loire', "Provence-Alpes-Côte d'Azur", 'Île-de-France']
{'Bretagne': 1759, 'Nouvelle-Aquitaine': 6038, 'Grand Est': 592, 'Auvergne-Rhône-Alpes': 322, 'Île-de-France': 3032, 'Occitanie': 751, 'Pays de la Loire': 917, 'Normandie': 2681, "Provence-Alpes-Côte d'Azur": 2160, 'Centre-Val de Loire': 44, 'Corse': 70, 'Hauts-de-France': 46, 'Bourgogne-Franche-Comté': 41, 'Guadeloupe': 0, 'Martinique': 0, 'Guyane': 0, 'La Réunion': 0, 'Mayotte': 0}


Map(center=[46.42252978589571, 2.99580993652344], controls=(AttributionControl(options=['position', 'prefix'],…

In [79]:
data, countNull = analyse_tweetsCount("Italy")
print(data)
label = Label()

colorScheme = ["#FFFCFC", "#F1DCDC","#DB8585", "#BB1216","#6c0000"]
colormap_dept = color.StepColormap(colors=colorScheme,vmin=min(data.values()),vmax=max(data.values()),index=[0, 100, 1000, 10000, 50000,100000])
legend = LegendControl({"0 - 100":colorScheme[0], "100 - 1K":colorScheme[1],"1K - 10K":colorScheme[2],"10K - 50K":colorScheme[3],"50K+": colorScheme[4]},fontsize=2, name="Tweets count", position="bottomleft")

m = Map(layers=(), center=( 41.9952978589571, 12.90),zoom_control=False, zoom=6)

layer = ipyleaflet.Choropleth(
    geo_data=italy_data,
    choro_data=data,
    key_on = "id",
    colormap=colormap_dept,
    style={'fillOpacity': 0.9}
)

layer.on_click(my_method)
m.on_interaction(handle_interaction)
m.add_layer(layer)
m.add_control(legend)
m.layout.width = '63%'
m.layout.height = '660px'
m


['Abruzzo', 'Basilicata', 'Calabria', 'Campania', 'Emilia-Romagna', 'Friuli-Venezia Giulia', 'Lazio', 'Liguria', 'Lombardia', 'Marche', 'Molise', 'Piemonte', 'Puglia', 'Sardegna', 'Sicilia', 'Toscana', 'Trentino-Alto Adige', 'Umbria', "Valle d'Aosta", 'Veneto']
{'Lombardia': 1109, 'Sardegna': 950, 'Emilia-Romagna': 473, 'Lazio': 343, 'Sicilia': 520, 'Liguria': 22, 'Toscana': 76, 'Veneto': 463, 'Calabria': 366, 'Piemonte': 92, 'Puglia': 349, 'Abruzzo': 16, 'Marche': 113, 'Trentino-Alto Adige': 16, 'Campania': 65, 'Friuli-Venezia Giulia': 17, 'Umbria': 35, 'Basilicata': 20, "Valle d'Aosta": 1, 'Molise': 0}


Map(center=[41.9952978589571, 12.9], controls=(AttributionControl(options=['position', 'prefix'], position='bo…

In [80]:
data, countNull = analyse_tweetsCount("India")
print(data)
label = Label()

colorScheme = ["#FFFCFC", "#F1DCDC","#DB8585", "#BB1216","#6c0000"]
colormap_dept = color.StepColormap(colors=colorScheme,vmin=min(data.values()),vmax=max(data.values()),index=[0, 100, 1000, 10000, 50000,100000])
legend = LegendControl({"0 - 100":colorScheme[0], "100 - 1K":colorScheme[1],"1K - 10K":colorScheme[2],"10K - 50K":colorScheme[3],"50K+": colorScheme[4]},fontsize=2, name="Tweets count", position="bottomleft")

m = Map(layers=(), center=( 23.2252978589571, 84.44580993652344), zoom_control=False,zoom=5)

layer = ipyleaflet.Choropleth(
    geo_data=india_data,
    choro_data=data,
    key_on = "id",
    colormap=colormap_dept,
    style={'fillOpacity': 0.9}
)

layer.on_click(my_method)
m.on_interaction(handle_interaction)
m.add_layer(layer)
m.add_control(legend)
m.layout.width = '76%'
m.layout.height = '750px'
m


['Andaman & Nicobar Island', 'Andhra Pradesh', 'Arunanchal Pradesh', 'Assam', 'Bihar', 'Chandigarh', 'Chhattisgarh', 'Dadara & Nagar Havelli', 'Daman & Diu', 'Goa', 'Gujarat', 'Haryana', 'Himachal Pradesh', 'Jammu & Kashmir', 'Jharkhand', 'Karnataka', 'Kerala', 'Lakshadweep', 'Madhya Pradesh', 'Maharashtra', 'Manipur', 'Meghalaya', 'Mizoram', 'NCT of Delhi', 'Nagaland', 'Odisha', 'Puducherry', 'Punjab', 'Rajasthan', 'Sikkim', 'Tamil Nadu', 'Telangana', 'Tripura', 'Uttar Pradesh', 'Uttarakhand', 'West Bengal']
{'Madhya Pradesh': 67015, 'NCT of Delhi': 8485, 'Jammu & Kashmir': 9600, 'Maharashtra': 35111, 'Rajasthan': 78271, 'Uttar Pradesh': 12497, 'Haryana': 1888, 'Tamil Nadu': 9521, 'Gujarat': 18192, 'Karnataka': 13557, 'Chandigarh': 759, 'Andhra Pradesh': 1661, 'Telangana': 2965, 'West Bengal': 797, 'Chhattisgarh': 632, 'Kerala': 1636, 'Uttarakhand': 120, 'Punjab': 829, 'Goa': 35, 'Tripura': 6, 'Himachal Pradesh': 8, 'Odisha': 45, 'Laccadives': 101, 'Jharkhand': 71, 'Assam': 20, 'Biha

Map(center=[23.2252978589571, 84.44580993652345], controls=(AttributionControl(options=['position', 'prefix'],…

In [81]:
label = Label()
data, countNull = analyse_tweetsCount("United States")
colorScheme = ["#FFFCFC", "#F1DCDC","#DB8585", "#BB1216","#6c0000"]
colormap_dept = color.StepColormap(colors=colorScheme,vmin=min(data.values()),vmax=max(data.values()),index=[0, 100, 1000, 10000, 50000,100000])
legend = LegendControl({"0 - 100":colorScheme[0], "100 - 1K":colorScheme[1],"1K - 10K":colorScheme[2],"10K - 50K":colorScheme[3],"50K+": colorScheme[4]},fontsize=2, name="Tweets count", position="bottomleft")

m = Map(layers=(), center=( 37.6252978589571, -94.24580993652344),zoom_control=False,zoom=4)
#
layer = ipyleaflet.Choropleth(geo_data=us_data,choro_data=data,key_on = "id",colormap=colormap_dept,style={'fillOpacity': 0.9}
)
layer.on_click(my_method)
m.on_interaction(handle_interaction)
m.add_layer(layer)
m.add_control(legend)
m.layout.width = '75%'
m.layout.height = '440px'
m


['Alabama', 'Alaska', 'Arizona', 'Arkansas', 'California', 'Colorado', 'Connecticut', 'Delaware', 'District of Columbia', 'Florida', 'Georgia', 'Hawaii', 'Idaho', 'Illinois', 'Indiana', 'Iowa', 'Kansas', 'Kentucky', 'Louisiana', 'Maine', 'Maryland', 'Massachusetts', 'Michigan', 'Minnesota', 'Mississippi', 'Missouri', 'Montana', 'Nebraska', 'Nevada', 'New Hampshire', 'New Jersey', 'New Mexico', 'New York', 'North Carolina', 'North Dakota', 'Ohio', 'Oklahoma', 'Oregon', 'Pennsylvania', 'Puerto Rico', 'Rhode Island', 'South Carolina', 'South Dakota', 'Tennessee', 'Texas', 'Utah', 'Vermont', 'Virginia', 'Washington', 'West Virginia', 'Wisconsin', 'Wyoming']


Map(center=[37.6252978589571, -94.24580993652344], controls=(AttributionControl(options=['position', 'prefix']…

In [82]:
label = Label()
data, countNull = analyse_tweetsCount("United Kingdom")
colorScheme = ["#FFFCFC", "#F1DCDC","#DB8585", "#BB1216","#6c0000"]
colormap_dept = color.StepColormap(colors=colorScheme,vmin=min(data.values()),vmax=max(data.values()),index=[0, 100, 1000, 10000, 50000,100000])
legend = LegendControl({"0 - 100":colorScheme[0], "100 - 1K":colorScheme[1],"1K - 10K":colorScheme[2],"10K - 50K":colorScheme[3],"50K+": colorScheme[4]},fontsize=2, name="Tweets count", position="bottomleft")

m = Map(layers=(), center=( 54.9, -2.74580993652344),zoom_control=False,zoom=6)
layer = ipyleaflet.Choropleth(geo_data= uk_data,choro_data=data,key_on="id",colormap=colormap_dept,style={'fillOpacity': 1}
)

layer.on_click(set_popUp)
m.on_interaction(handle_interaction_country)
m.add_layer(layer)
m.add_control(legend)
m.layout.width = '60%'
m.layout.height = '750px'
m


['Aberdeen', 'Aberdeenshire', 'Anglesey', 'Angus', 'Antrim', 'Ards', 'Argyll and Bute', 'Armagh', 'Ballymena', 'Ballymoney', 'Banbridge', 'Barking and Dagenham', 'Bath and North East Somerset', 'Bedfordshire', 'Belfast', 'Berkshire', 'Bexley', 'Blackburn with Darwen', 'Blaenau Gwent', 'Bournemouth', 'Brent', 'Bridgend', 'Brighton and Hove', 'Bristol', 'Bromley', 'Buckinghamshire', 'Caerphilly', 'Cambridgeshire', 'Camden', 'Cardiff', 'Carmarthenshire', 'Carrickfergus', 'Castlereagh', 'Ceredigion', 'Cheshire', 'Clackmannanshire', 'Coleraine', 'Conwy', 'Cookstown', 'Cornwall', 'Craigavon', 'Croydon', 'Cumbria', 'Darlington', 'Denbighshire', 'Derby', 'Derbyshire', 'Derry', 'Devon', 'Dorset', 'Down', 'Dumfries and Galloway', 'Dundee', 'Dungannon', 'Durham', 'Ealing', 'East Ayrshire', 'East Dunbartonshire', 'East Lothian', 'East Renfrewshire', 'East Riding of Yorkshire', 'East Sussex', 'Edinburgh', 'Eilean Siar', 'Enfield', 'Essex', 'Falkirk', 'Fermanagh', 'Fife', 'Flintshire', 'Glasgow', '

Map(center=[54.9, -2.74580993652344], controls=(AttributionControl(options=['position', 'prefix'], position='b…